In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_dir = "/content/drive/MyDrive/Sayem_Potato_Leaf_/PlantVillage2/Training"
val_dar = "/content/drive/MyDrive/Sayem_Potato_Leaf_/PlantVillage2/Validation"

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  # validation_split=0.25,
  # subset="training",
  # seed=123,
  image_size=(128,128),
  batch_size=64)

val_ds = tf.keras.utils.image_dataset_from_directory(
  val_dar,
  # validation_split=0.25,
  # subset="validation",
  # seed=123,
  image_size=(128,128),
  batch_size=64
  )

Found 3251 files belonging to 3 classes.
Found 416 files belonging to 3 classes.


In [ ]:
vgg = VGG16(input_shape = (128, 128, 3), weights = 'imagenet', include_top = False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(3, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the number of folds (k)
k = 5

# Initialize KFold cross-validation
kf = KFold(n_splits=k, shuffle=True)

# Initialize lists to store evaluation metrics
accuracy_scores = []
loss_scores = []

# Perform K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(train_ds)):
    train_data = train_ds.take(train_index)
    val_data = train_ds.take(val_index)

    # Define checkpoint callback to save the best model
    checkpoint_path = f"/content/drive/MyDrive/Road_To_BCS/vgg16.h5" #checkpoint path ba model save er path deya lagbe

    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

    # Compile the model
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    # Train the model with checkpoint callback
    history = model.fit(train_data, validation_data=val_data, epochs=15, callbacks=[checkpoint], shuffle=True)

    # Evaluate the best model on the validation data
    best_model = tf.keras.models.load_model(checkpoint_path)
    loss, accuracy = best_model.evaluate(val_data)

    # Append evaluation metrics to the lists
    loss_scores.append(loss)
    accuracy_scores.append(accuracy)

# Calculate and print average evaluation metrics
print("Average Loss:", np.mean(loss_scores))
print("Average Accuracy:", np.mean(accuracy_scores))

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the number of folds (k)
k = 5

# Initialize KFold cross-validation
kf = KFold(n_splits=k, shuffle=True)

# Initialize lists to store evaluation metrics
accuracy_scores = []
loss_scores = []

# Convert datasets to arrays for compatibility with KFold.split
train_ds_array = np.array(list(train_ds.as_numpy_iterator()))
val_ds_array = np.array(list(val_ds.as_numpy_iterator()))

# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(train_ds_array):
    train_data = train_ds_array[train_index]
    val_data = train_ds_array[val_index]

    # Define checkpoint callback to save the best model
    checkpoint_path = f"/content/drive/MyDrive/Road_To_BCS/vgg16.h5"
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

    # Train the model with checkpoint callback
    history = model.fit(train_data[0], train_data[1], validation_data=(val_data[0], val_data[1]), epochs=15, callbacks=[checkpoint], shuffle=True)

    # Evaluate the best model on the validation data
    best_model = tf.keras.models.load_model(checkpoint_path)
    loss, accuracy = best_model.evaluate(val_data[0], val_data[1])

    # Append evaluation metrics to the lists
    loss_scores.append(loss)
    accuracy_scores.append(accuracy)

# Calculate and print average evaluation metrics
print("Average Loss:", np.mean(loss_scores))
print("Average Accuracy:", np.mean(accuracy_scores))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (51, 2) + inhomogeneous part.

In [ ]:
history = model.fit(train_ds, validation_data = val_ds, epochs = 15, batch_size = 64,
                    shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label = 'validation_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.1, 1.2])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label = 'validation_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 6])
plt.legend(loc='upper right')

In [ ]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

all_val_labels = []
all_val_predictions = []

# Evaluate the model on the validation dataset batch by batch
for batch in val_ds:
    val_images, val_labels_batch = batch
    val_predictions_batch = model.predict(val_images)

    val_labels_batch = np.array(val_labels_batch)
    val_predictions_batch = np.argmax(val_predictions_batch, axis=-1)

    all_val_labels.extend(val_labels_batch)
    all_val_predictions.extend(val_predictions_batch)

# Calculate the confusion matrix for all validation data
conf_matrix = confusion_matrix(all_val_labels, all_val_predictions)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Oranges', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix Heatmap')
plt.show()

In [ ]:
print(classification_report(all_val_labels, all_val_predictions, target_names=class_names))